# Load Data

In [2]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np
#import pandas as pd

%load_ext autoreload
%autoreload 2

oy, otx, oids = load_csv_data('Data/train.csv', False)
otest_y, otest_tx, otest_ids= load_csv_data('Data/test.csv', False)

#data = pd.read_csv('Data/train.csv', sep=",")


In [3]:
#pd.plotting.scatter_matrix(data)

In [14]:
def prepro(M,c):
    return addFunckyThings(addColumns(M, c), M)

In [15]:
from preprocessing import *

# Finding the Best columns
best_number_of_colums = np.zeros(4)
best_per_of_columns = np.zeros(4)

for nbColumns in range(10,13):
#for nbColumns in [2, 5, 10]:
    y, tx, ids = oy, otx, oids
    test_y, test_tx,test_ids = otest_y, otest_tx, otest_ids
    # Preprocessing


    PRI_jet_num_colomn_train = tx[:, 22]
    PRI_jet_num_colomn_test = test_tx[:, 22]


    y_clustered = list()
    tx_clustered = list()
    ids_clustered = list()
    test_y_clustered = list()
    test_tx_clustered = list()
    test_ids_clustered = list()
   

    for i in range(4):
        indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]
        

        y_clustered.append(y[indices])
        tx_clustered.append(tx[indices])
        ids_clustered.append(ids[indices])

        test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]
        
        test_y_clustered.append(test_y[test_indices])
        test_tx_clustered.append(test_tx[test_indices])
        test_ids_clustered.append(test_ids[test_indices])

        #delete colinear columns and preprocess the data
        indices_to_delete = list()
        for col in range(tx_clustered[i].shape[1]):
            if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
                indices_to_delete.append(col)

        deleted_tx = np.delete(tx_clustered[i], indices_to_delete, 1)
        tx_clustered[i] = prepro(deleted_tx, nbColumns)
        
        deleted_tx = np.delete(test_tx_clustered[i], indices_to_delete, 1)
        test_tx_clustered[i] = prepro(deleted_tx, nbColumns)


    # Cross-Validation & Weight computation

    from cross_validation import *
    w = list()
    result = list()
    

    loss = 0
    perGood = 0
    maxW = 0
    for i in range(4):
        wi,loss_te, perGoodI = cross_validation_run(tx_clustered[i], y_clustered[i])
        if(perGoodI > best_per_of_columns[i]):
            best_per_of_columns[i] = perGoodI
            best_number_of_colums[i] = nbColumns
        
        w.append(wi)
        maxW += np.max(np.abs(wi))/4
        loss += loss_te/4
        perGood += perGoodI/4
        test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])
        
    print("Number of columns : ", nbColumns, " loss : ", loss, "Max of w : ", maxW,  " Percentage of true Y : ", perGood)
print(best_number_of_colums) 

best lambda:  6.21016941892e-07 best perc:  0.84037953399
best lambda:  1.26896100317e-06 best perc:  0.841620626151
best lambda:  2.21221629107e-05 best perc:  0.846825206181
best lambda:  6.21016941892e-07 best perc:  0.851629433902


LinAlgError: Singular matrix

In [ ]:
print(best_per_of_columns)
print(best_number_of_colums) 

# Using least square on [cst, tx, tx^2, ...] 
31 - 0.695085425438 

61 - 0.776

91 - 0.787

121 - 0.796

151 - 0.805

181 - 0.807

211 - 0.809

241 - 0.816

271 - 0.822

# Computing with good columns

In [ ]:
y, tx, ids = oy, otx, oids
test_y, test_tx,test_ids = otest_y, otest_tx, otest_ids
# Preprocessing


PRI_jet_num_colomn_train = tx[:, 22]
PRI_jet_num_colomn_test = test_tx[:, 22]


y_clustered = list()
tx_clustered = list()
ids_clustered = list()
test_y_clustered = list()
test_tx_clustered = list()
test_ids_clustered = list()


for i in range(4):
    indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]
 

    y_clustered.append(y[indices])
    tx_clustered.append(tx[indices])
    ids_clustered.append(ids[indices])

    test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]

    test_y_clustered.append(test_y[test_indices])
    test_tx_clustered.append(test_tx[test_indices])
    test_ids_clustered.append(test_ids[test_indices])

    #delete colinear columns and preprocess the data
    indices_to_delete = list()
    for col in range(tx_clustered[i].shape[1]):
        if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
            indices_to_delete.append(col)

    deleted_tx = np.delete(tx_clustered[i], indices_to_delete, 1)
    tx_clustered[i] = prepro(deleted_tx, int(best_number_of_colums[i]))#addColumns(deleted_tx, int(best_number_of_colums[i]))

    deleted_tx = np.delete(test_tx_clustered[i], indices_to_delete, 1)
    test_tx_clustered[i] = prepro(deleted_tx, int(best_number_of_colums[i]))#addColumns(deleted_tx, int(best_number_of_colums[i]))


# Cross-Validation & Weight computation

from cross_validation import *
w = list()
result = list()


loss = 0
perGood = 0
maxW = 0
for i in range(4):
    wi,loss_te, perGoodI = cross_validation_run(tx_clustered[i], y_clustered[i])
    if(perGoodI > best_per_of_columns[i]):
        best_per_of_columns[i] = perGoodI
        best_number_of_colums[i] = nbColumns

    w.append(wi)
    maxW += np.max(np.abs(wi))/4
    loss += loss_te/4
    perGood += perGoodI/4
    test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])

print(" loss : ", loss, "Max of w : ", maxW,  " Percentage of true Y : ", perGood)

old best: 0.826606619791

new best: 0.827178252354

intermediate best: 0.830793909724 (np.logspace(-5, 0, 15))



# Analysing why it isn't working

In [ ]:

for i in range(4):
    realPred = np.dot(tx_clustered[i], w[i])
    #print(np.max(realPred))
    #df = pd.DataFrame(realPred)
    #df.hist(bins=300)
    y_pred = predict_labels(w[i],tx_clustered[i])
    
    false_index = [ind for ind,a in enumerate(y_clustered[i]) if y_pred[ind] != a]
    
    print(np.mean(realPred[false_index]))
    print("Per : ", percentageGood(y_clustered[i],tx_clustered[i],w[i]), "Len : ", len(y_clustered[i]) ) 
    
    print(len(w[i]))


# Post Processing

In [11]:
test_ids = [item for sublist in test_ids_clustered for item in sublist]
y_pred = [item for sublist in test_y_clustered for item in sublist]

test_ids, y_pred = zip(*sorted(zip(test_ids, y_pred)))

# Output Data

In [12]:
name = 'whole_matrix_submission.csv'
create_csv_submission(test_ids, y_pred, name)

In [13]:
np.zeros(4)

array([ 0.,  0.,  0.,  0.])